#UE03 - SPARQL Query

Please complete the 10 tasks in the `2. SPARQL` sheet of `SemAI.jar` first, and then transfer the task descriptions and your solutiosn in executable form to this notebook.

## Preparation

Reuse imports and functions from https://github.com/jku-win-dke/SemAI/blob/main/V02_SPARQL.ipynb and load the solar system graph. Query the solar system graph to check that everything works fine. 

In [2]:
# Install required packages
!pip install -q rdflib

# Imports
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace
from rdflib.namespace import FOAF , XSD , RDFS, NamespaceManager 

# Convenient Functions
def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = []                             # a list of dictionaries, as intermediate format to construct the pandas DataFrame
  for result in results:                # iterate over the result set of the query, a result is an instance of rdflib.query.ResultRow
    row = {}                            #     create a dictionary to hold a single row of the result
    for var in results.vars:            #     iterate over the variables of the SPARQLResult to add a dictionary entry for each variable
      if (isinstance(result[var],URIRef) and use_prefixes):
        row[var] = result[var].n3(graph.namespace_manager)   # use namespace prefixes to shorten URIs
      else:
        row[var] = result[var]                  
    rows.append(row)                    #     add the dictionary (row) to the list 
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def sparql_construct(graph, query):
  result_graph = Graph(namespace_manager = g.namespace_manager)  # create a Graph object that reuses the namespace prefixes of the original graph
  result_graph += graph.query(query)                             # execute the construct query against the original graph and add the resulting graph to the new one
  return result_graph

def sparql_ask(graph, query):
  return bool(graph.query(query))      # an ASK query has a boolean result, which should be returned as such

# Load solar system graph
g = Graph()
g.parse("https://raw.githubusercontent.com/jku-win-dke/SemAI/main/data/solarsystem.ttl",format="turtle")

# Query solar system graph (to check that everything works fine)
df = sparql_select(g,"""
  SELECT ?planet ?apoapsis ?apoapsis_uom
  WHERE { 
    ?planet rdf:type dbo:Planet . 
    OPTIONAL { ?planet v:apoapsis [rdf:value ?apoapsis ; v:uom ?apoapsis_uom ].  }
  }
""")
df

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 KB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 4.9 MB/s eta 0:00:00


,planet,apoapsis,apoapsis_uom
0,:Mercury,0.467,unit:AU
1,:Venus,0.728,unit:AU
2,:Earth,1.017,unit:AU
3,:Earth,149597871,unit:KM
4,:Mars,1.666,unit:AU
5,:Jupiter,5.4588,unit:AU
6,:Saturn,9.0412,unit:AU
7,:Uranus,20.11,unit:AU
8,:Neptune,30.33,unit:AU



## Task 1 (1 pt)

Geben Sie alle Zwergplaneten (Instanzen der Klasse dbo:DwarfPlanet) aus und falls vorhanden deren genaue Entsprechung (skos:exactMatch). Ordnen Sie das Ergebnis aufsteigend nach den URIs der Zwergplaneten.

In [ ]:
df = sparql_select(g,"""
SELECT ?d ?match
WHERE { 
  ?d a dbo:DwarfPlanet. 
  OPTIONAL { ?d skos:exactMatch ?match.}  
}
ORDER BY ?d 
""")
df


## Task 2 (1 pt)
Ermitteln Sie alle Sterne, ihr exactMatch, und ihre Masse (geben Sie den Wert und die Maßeinheit aus). Ordnen Sie das Ergebnis nach der URI der Sterne

In [ ]:
df = sparql_select(g,"""
SELECT ?star ?match ?massVal ?massUoM
WHERE { 
  ?star a dbo:Star. 
  ?star skos:exactMatch ?match. 
  ?star v:mass[rdf:value ?massVal; v:uom ?massUoM]. 
}
ORDER BY ?star
""")
df



## Task 3 (1 pt)

Die Planeten unseres Sonnensystems und ihre jeweilige Anzahl an Monden. Unterscheiden Sie die im RDF-Graph beschriebenen Monde und die im RDF-Graph erfasste Anzahl von Monden. Sortieren Sie nach den Planeten.

In [ ]:
df = sparql_select(g,"""
SELECT ?planet ?assertedNo (count(?moon) AS ?noOfDescribedMoons) 
WHERE { 
  ?planet a dbo:Planet. 
  ?planet v:nrOfMoons ?assertedNo.
  OPTIONAL {?moon v:orbits ?planet.}
}
GROUP BY ?planet ?assertedNo
ORDER BY ?planet""")
df


## Task 4 (1 pt)


Ermitteln sie für die Planeten in unserem Sonnensystem die durchschnittliche Anzahl an Monden (die auch im RDF-Graph beschrieben sind) pro Planet.

In [ ]:
df = sparql_select(g,"""
SELECT (avg(?noOfDescribedMoons) AS ?avgNoOfDescribedMoons)
{
  SELECT ?planet (count(?moon) AS ?noOfDescribedMoons)
  WHERE{
    ?planet a dbo:Planet
    OPTIONAL { ?moon v:orbits ?planet }
  } GROUP BY ?planet
}
""")
df

## Task 5 (1 pt)

Geben Sie die im RDF-Graph verwendeten Klassen und ihre Anzahl an Instanzen aus. Geben Sie nur Klassen mit mindestens 2 Instanzen aus. Ordnen Sie die Ausgabe nach der URI der Klassen.

In [ ]:
df = sparql_select(g,"""
SELECT ?class (count(?type) AS ?noOfInstances)
WHERE { 
  ?type a ?class.
}
GROUP BY ?class
HAVING (count(?type) > 1)
ORDER BY ?class
""")
df

**Kommentar: In SemAI.jar HAVING (?noOfInstances > 1) möglich. Hier leider nicht.
Original in SemAI.jar:**

```
SELECT ?class (count(?type) AS ?noOfInstances)
WHERE { 
  ?type a ?class.
}
GROUP BY ?class
HAVING ( ?noOfInstances > 1)
ORDER BY ?class
```



## Task 6 (1 pt)

Geben Sie die im RDF-Graph verwendeten Properties und ihre Häufigkeit je Klasse aus. Gezählt werden sollen die Instanzen der Klasse, die die Property ausprägen. Geben Sie nur Properties zu Klassen aus, die von mindestens 7 Instanzen dieser Klasse ausgeprägt werden. Ordnen Sie die Ausgabe zuerst nach der Klasse und dann nach der Property.

In [ ]:
df = sparql_select(g,"""
SELECT ?class ?prop (count(DISTINCT ?x) AS ?noOfInstances)
WHERE {
?x a ?class.
?x ?prop ?y.
} 
GROUP BY ?class ?prop
HAVING(count(?x) >= 7)
ORDER BY ?class ?prop
""")
df

## Task 7 (1 pt)

Ermitteln Sie alle Sterne, sowie alle Monde. Zu jedem Mond ermitteln Sie auch den Planet, den dieser Mond umrundet. Ordnen Sie die Ausgabe nach der URI der Sterne und Monde.

In [ ]:
df = sparql_select(g,"""
SELECT ?x ?y
WHERE { 
   {?x a dbo:Star} UNION {?x a dbo:Satellite}.
   OPTIONAL {?x v:orbits ?y}.
}
ORDER BY ?x
""")
df

## Task 8 (1 pt)

Erzeugen sie mittels einer Construct-Query den angezeigten RDF-GRAPH (Expected Result). Reihenfolge ist unerheblich.

In [3]:
df = sparql_construct(g,"""
CONSTRUCT {
  ?x rdf:type v:Himmelskoerper.
  ?x v:wirdUmrundetVon ?y. 
  ?z v:wirdUmrundetVon ?t.   
}
WHERE { 
  {?x a dbo:Planet} UNION {?x a dbo:Satellite}.
  OPTIONAL {?y v:orbits ?x}.
  OPTIONAL {?z skos:exactMatch dbr:Sun}.
  OPTIONAL {?t a dbo:Planet; v:orbits ?z}.  
}
""")
print(df.serialize(format="turtle"))

@prefix : <http://dke.jku.at/example/solarsystem/> .
@prefix v: <http://dke.jku.at/example/examplevocabulary/> .

:Sun v:wirdUmrundetVon :Earth,
        :Jupiter,
        :Mars,
        :Mercury,
        :Neptune,
        :Saturn,
        :Uranus,
        :Venus .

:Callisto a v:Himmelskoerper .

:Deimos a v:Himmelskoerper .

:Earth a v:Himmelskoerper ;
    v:wirdUmrundetVon :Moon .

:Enceladus a v:Himmelskoerper .

:Europa a v:Himmelskoerper .

:Ganymede a v:Himmelskoerper .

:Io a v:Himmelskoerper .

:Jupiter a v:Himmelskoerper ;
    v:wirdUmrundetVon :Callisto,
        :Europa,
        :Ganymede,
        :Io .

:Mars a v:Himmelskoerper ;
    v:wirdUmrundetVon :Deimos,
        :Phobos .

:Mercury a v:Himmelskoerper .

:Mimas a v:Himmelskoerper .

:Moon a v:Himmelskoerper .

:Neptune a v:Himmelskoerper .

:Phobos a v:Himmelskoerper .

:Saturn a v:Himmelskoerper ;
    v:wirdUmrundetVon :Enceladus,
        :Mimas .

:Uranus a v:Himmelskoerper .

:Venus a v:Himmelskoerper .




## Task 9 (1 pt)
Gibt es einen Stern im RDF-Graph, der massereicher als die Sonne ist.

In [ ]:
df = sparql_ask(g,"""
ASK {    
  {SELECT ?x 
   WHERE { 
     FILTER (?val1 > ?val2 || ?val1 > 0.9942000004971).
     {SELECT ?val1
      WHERE {
        ?x v:mass[rdf:value ?val1; v:uom ?massUoM1]. 
        ?x a dbo:Star.
      }}.
     {SELECT ?val2 
      WHERE{ :Sun v:mass[rdf:value ?val2; v:uom ?massUoM2]
      }}.
}}
}
""")
df

## Task 10 (1 pt)

Geben Sie eine Beschreibung der Resourcen aus, die einen Durchmesser zwischen 20.000 km und 30.000 km haben.

In [4]:
df = sparql_construct(g,"""
DESCRIBE ?x
WHERE {
?x a ?y ;
v:radius [rdf:value ?val; v:uom ?uom]
FILTER(?val >= 20000 && ?val <= 30000 && ?uom in (unit:KM))
}
""")
print(df.serialize(format="turtle"))

@prefix : <http://dke.jku.at/example/solarsystem/> .
@prefix dbo: <http://dbpedia.org/ontology/> .
@prefix dbr: <http://dbpedia.org/resource/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix unit: <http://qudt.org/vocab/unit/> .
@prefix v: <http://dke.jku.at/example/examplevocabulary/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

:Neptune a dbo:Planet ;
    v:apoapsis [ v:uom unit:AU ;
            rdf:value 30.33 ] ;
    v:nrOfMoons 14 ;
    v:orbitalPeriod [ v:uom unit:YR ;
            rdf:value 164.8 ] ;
    v:orbits :Sun ;
    v:radius [ v:uom unit:KM ;
            rdf:value 24622 ] ;
    v:temperature [ v:uom unit:Deg_C ;
            rdf:value -201 ] ;
    skos:exactMatch dbr:Neptune .

:Uranus a dbo:Planet ;
    v:apoapsis [ v:uom unit:AU ;
            rdf:value 20.11 ] ;
    v:nrOfMoons 27 ;
    v:orbitalPeriod [ v:uom unit:YR ;
            rdf:value 84.0205 ] ;
    v:orbits :Sun ;
    v:radius